In [1]:
import numpy as np
import pandas as pd

In [56]:
df_path = '/Users/Briana/Documents/Grad_School/HPS/comparison_populations/SDSS_MZR/'
andrews = pd.read_csv(df_path+'andrews_flux_table.csv')
andrews = andrews[~np.isnan(andrews['OII3727']) & ~np.isnan(andrews['OIII5007']) & ~np.isnan(andrews['Ha'])]
#andrew = andrew.loc[:,['']]
andrews = andrews.sort_values(by=['Ngal'], ascending=False).head(30).loc[:,['OII3727','e_OII3727','OIII5007','e_OIII5007','Ha','e_Ha','NII6583','e_NII6583' ]]
andrews['Hb'] = 100.0
andrews['e_Hb'] = 1.0

print(andrews.columns.values)

['OII3727' 'e_OII3727' 'OIII5007' 'e_OIII5007' 'Ha' 'e_Ha' 'NII6583'
 'e_NII6583' 'Hb' 'e_Hb']


## Build Reddened Spectra

In [57]:
#Functions defining the reddening

#define Calzetti law 2000 
#wl in angstroms
def calzetti(wl):
    wl = wl/10000.
    if wl <= 6300:
        return 2.695*(-2.156+(1.509/wl)-(0.189/(wl**2))+(0.011/(wl**3)))+4.05
    else:
        return 2.659*(-1.857+(1.040/wl))+4.05

def reddening(wl, E_bv):
    return 10**(-0.4*E_bv*calzetti(wl))

def reddening_err(wl, E_bv, E_bv_e):
    dr = np.sqrt((0.4*calzetti(wl)*np.log(10)*(10**(-0.4*E_bv*calzetti(wl)))*E_bv_e)**2)
    return dr

print 2.44*reddening(5007, 0.295)

0.70563988303


In [68]:
#reddeing parameters 
mu = 0.295
si = 0.165

red = mu+si

In [69]:
and_c = andrews.copy()

and_c['OII3727'] = andrews['OII3727']*reddening(3727,red)
and_c['e_OII3727'] = andrews['e_OII3727']*reddening(3727,red)
and_c['OIII5007'] = andrews['OIII5007']*reddening(5007,red)
and_c['e_OIII5007'] = andrews['e_OIII5007']*reddening(5007,red)
and_c['Ha'] = andrews['Ha']*reddening(6562,red)
and_c['e_Ha'] = andrews['e_Ha']*reddening(6562,red)
and_c['NII6583'] = andrews['NII6583']*reddening(6583,red)
and_c['e_NII6583'] = andrews['e_NII6583']*reddening(6583,red)
and_c['Hb'] = andrews['Hb']*reddening(4861,red)
and_c['e_Hb'] = andrews['e_Hb']*reddening(4861,red)

In [70]:
andrews.describe()

,OII3727,e_OII3727,OIII5007,e_OIII5007,Ha,e_Ha,NII6583,e_NII6583,Hb,e_Hb
count,30.000000,30.00000,30.000000,30.000000,30.0,30.000000,30.000000,30.000000,30.0,30.0
mean,237.487333,0.75800,54.739667,0.413333,286.0,0.829000,99.131000,0.840000,100.0,1.0
std,47.602558,0.26501,34.575270,0.064878,0.0,0.504923,16.712814,0.495037,0.0,0.0
min,187.360000,0.41000,30.940000,0.320000,286.0,0.420000,57.600000,0.430000,100.0,1.0
25%,199.777500,0.50250,33.687500,0.370000,286.0,0.542500,90.762500,0.550000,100.0,1.0
50%,216.120000,0.66000,37.480000,0.395000,286.0,0.610000,105.320000,0.625000,100.0,1.0
75%,268.395000,0.95500,59.897500,0.440000,286.0,0.760000,111.525000,0.787500,100.0,1.0
max,337.920000,1.29000,161.620000,0.600000,286.0,2.330000,119.580000,2.270000,100.0,1.0


In [71]:
and_c.describe()

,OII3727,e_OII3727,OIII5007,e_OIII5007,Ha,e_Ha,NII6583,e_NII6583,Hb,e_Hb
count,30.000000,30.000000,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,3.000000e+01,3.000000e+01
mean,18.246207,0.058237,7.909135,0.059721,6.891359e+01,0.199753,24.020592,0.203542,1.360504e+01,1.360504e-01
std,3.657316,0.020361,4.995655,0.009374,5.781517e-14,0.121665,4.049709,0.119953,5.420172e-15,5.646013e-17
min,14.394913,0.031500,4.470408,0.046236,6.891359e+01,0.101202,13.957149,0.104194,1.360504e+01,1.360504e-01
25%,15.348952,0.038607,4.867384,0.053460,6.891359e+01,0.130719,21.992808,0.133271,1.360504e+01,1.360504e-01
50%,16.604550,0.050708,5.415349,0.057072,6.891359e+01,0.146984,25.520259,0.151445,1.360504e+01,1.360504e-01
75%,20.620851,0.073373,8.654371,0.063574,6.891359e+01,0.183127,27.023802,0.190820,1.360504e+01,1.360504e-01
max,25.962473,0.099111,23.351884,0.086692,6.891359e+01,0.561429,28.975622,0.550047,1.360504e+01,1.360504e-01


In [72]:
and_c.to_csv(df_path+'andrews_flux_table_red_'+str(red)+'.csv')

# Read Test Results

In [76]:
and_lo = pd.read_csv(df_path+'andrews_flux_table_red_0.13.csv')
and_lo_wRL = pd.read_csv(df_path+'andrews_flux_table_red_0.13_wRL.csv')

and_up = pd.read_csv(df_path+'andrews_flux_table_red_0.46.csv')
and_up_wRL = pd.read_csv(df_path+'andrews_flux_table_red_0.46_wRL.csv')

and_med = pd.read_csv(df_path+'andrews_flux_table_red_0.295.csv')
and_med_wRL = pd.read_csv(df_path+'andrews_flux_table_red_0.295_wRL.csv')

In [77]:
cols = ['lo', 'lo_wRL', 'up', 'up_wRL', 'med', 'med_wRL']
df = pd.DataFrame(columns=cols)
df['lo'] = and_lo['E(B-V)']
df['lo_wRL'] = and_lo_wRL['E(B-V)']
df['up'] = and_up['E(B-V)']
df['up_wRL'] = and_up_wRL['E(B-V)']
df['med'] = and_med['E(B-V)']
df['med_wRL'] = and_med_wRL['E(B-V)']

In [78]:
print(df)

          lo    lo_wRL        up    up_wRL       med   med_wRL
0   0.341972  0.429543  0.449905  0.520660  0.409507  0.480602
1   0.366223  0.434039  0.452764  0.537413  0.406607  0.484554
2   0.361786  0.428871  0.435959  0.514838  0.394478  0.479097
3   0.345640  0.417757  0.436195  0.517627  0.397321  0.468759
4   0.355032  0.438146  0.443215  0.521853  0.407243  0.491624
5   0.334557  0.420806  0.427670  0.500457  0.392720  0.473508
6   0.355438  0.432147  0.458218  0.537083  0.409572  0.480318
7   0.345864  0.416365  0.431687  0.513664  0.396293  0.457587
8   0.339742  0.406647  0.436042  0.507471  0.386545  0.465661
9   0.359588  0.442146  0.456952  0.531688  0.400562  0.475159
10  0.347844  0.403267  0.423527  0.505714  0.380122  0.462614
11  0.346804  0.409549  0.434009  0.512203  0.381129  0.463715
12  0.342950  0.421172  0.429518  0.502561  0.396913  0.470407
13  0.342497  0.402156  0.423670  0.499050  0.375871  0.450678
14  0.356572  0.424430  0.445538  0.516348  0.402247  0

In [79]:
df.describe()

,lo,lo_wRL,up,up_wRL,med,med_wRL
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,0.350780,0.421607,0.438440,0.515885,0.395978,0.472607
std,0.012545,0.015151,0.013641,0.015987,0.013163,0.013486
min,0.328308,0.385389,0.413899,0.489102,0.375871,0.443572
25%,0.342610,0.408756,0.424811,0.500983,0.383203,0.463011
50%,0.347324,0.424495,0.437680,0.516987,0.396603,0.473716
75%,0.361538,0.434365,0.449635,0.530095,0.407084,0.484161
max,0.373645,0.442146,0.458757,0.540624,0.417045,0.493756
